# Tactical Layer

In this notebook the tactical layer is detailed. In order to execute this notebook, first be sure to launch `01-Open-loop.ipynb`. Results are read from the file `Output/SymOut.sqlite`. 

This notebook is separated in multiple parts 

1. [Reading of open loop simulations](#reading_open_loop)

2. [Solution of the tactical strategy](#tactical_solution) which can be explained in two phases. 

    - Determination of [**yielding times**](#yielding_time) and **desired time headways**  
    - Creation of [reference headway signal](#reference_headway) along time designed to comunicate with the low level controller

<a id='reading_open_loop'></a>

## Reading open loop simulations

In this simulation should have been already finished. 

In [6]:
import os 

from sqlalchemy import create_engine, MetaData
from sqlalchemy import Table, Column, String, Integer, Float 
from sqlalchemy import insert, delete, select, case, and_

import pandas as pd
import numpy as np

from bokeh.plotting import figure, show
from bokeh.sampledata.iris import flowers
from bokeh.io import output_notebook
from bokeh.palettes import Viridis,Paired
output_notebook() 

Loading BokehJS ...

#### Open connection 

Opening the database connection and converting the data into a `DataFrame` format 

In [7]:
dir_path = os.getcwd()

engine_path = ('..','Output','SymOut.sqlite')
engine_name = os.path.join(os.path.sep,*engine_path)
engine_full_name = os.path.join(dir_path,*engine_path)
engine_call = 'sqlite://'+engine_name
engine = create_engine(engine_call)
metadata = MetaData()

try: 
    ltbstr = 'Loaded table in: '
    connection = engine.connect()    
    traj = Table('traj', metadata, autoload=True, autoload_with=engine)
    results = connection.execute(stmt)
    print(ltbstr, engine)
except:
    print('Unsuccesful load. Database could not be read')

stmt = select([traj])
results = connection.execute(stmt).fetchall()
column_names = traj.columns.keys()
trajDf = pd.DataFrame(results, columns = column_names)

Loaded table in:  Engine(sqlite:///../Output/SymOut.sqlite)


<a id='tactical_solution'></a>

## Tactical strategy solution

In order to solve the tactical strategy problem the following procedure is established: 

 - Find the $X_m$ and $T_m$ for the leader of CAV 
 - Create the table `headway` in charge of storing data for the operational layer
 - Define functions required for the tactical operation 
 - Execute tactical operation 

#### Determine $X_m, T_m$

In [8]:
trajLeader = trajDf[trajDf['id'] == 0]
leaderFlt = trajLeader[trajLeader['abs']<0]
Xm = leaderFlt.iloc[-1]['abs']
Tm = leaderFlt.iloc[-1]['ti']
merge_point = (Xm,Tm)
leaderFlt.iloc[-1]

ti         41.2
id            0
type        CAV
tron    In_main
voie          1
dst      998.75
abs       -1.25
vit          25
ldr           0
spc       31.25
vld          25
Name: 4116, dtype: object

#### Open dB Connection + Table `headway`

In [9]:
if os.path.isfile(engine_full_name):
    try:
        ltbstr = 'Loaded table in: '
        connection = engine.connect()        
        traj = Table('traj', metadata, autoload=True, autoload_with=engine)        
        closed = Table('closed', metadata, autoload=True, autoload_with=engine)
        headway = Table('headway', metadata, autoload=True, autoload_with=engine)
        control = Table('control', metadata, autoload=True, autoload_with=engine)
    except:
        ltbstr = 'Created table in: '
        traj = Table('traj', metadata,
                 Column('ti', Float()),
                 Column('id', Integer()),
                 Column('type', String(3)),
                 Column('tron', String(10)),
                 Column('voie', Integer()),
                 Column('dst', Float()),
                 Column('abs', Float()),
                 Column('vit', Float()),
                 Column('ldr', Integer()),
                 Column('spc', Float()),
                 Column('vld', Float()))        
        closed = Table('closed', metadata,
                 Column('ti', Float()),
                 Column('id', Integer()),
                 Column('type', String(3)),
                 Column('tron', String(10)),
                 Column('voie', Integer()),
                 Column('dst', Float()),
                 Column('abs', Float()),
                 Column('vit', Float()),
                 Column('ldr', Integer()),
                 Column('spc', Float()),
                 Column('vld', Float()))
        control = Table('control', metadata,
                 Column('ti', Float()),
                 Column('id', Integer()),
                 Column('type', String(3)),
                 Column('tron', String(10)),
                 Column('voie', Integer()),
                 Column('ctr', Float()),
                 Column('nit', Integer())) 
        headway = Table('headway', metadata,
                 Column('ti', Float()),
                 Column('id', Integer()),
                 Column('gapt', Float()))
        metadata.create_all(engine)
        connection = engine.connect()
    finally: 
        print(ltbstr, engine)
                

Loaded table in:  Engine(sqlite:///../Output/SymOut.sqlite)


Since `leader` id is dynamic the following lines aims to freeze the leader information 

In [10]:
network_data = trajDf[(trajDf['ti']==20) & (trajDf['type']=='CAV')]
key = network_data.id.values
ldr = network_data.ldr.values
idx = range(len(key))
dveh_ldr = dict(zip(key,ldr))
dveh_idx = dict(zip(key,idx))

### 2a. Tactical operations 

#### Tactical: Functions

1. Traffic parameters 
2. Project on the congestion wave `find_projection`
3. Determine headways according to the vehicle type `find_headway`
4. Find different orders `find_variations`
5. Find final time headways for a particular ordered sequence `find_ref_hwy'
6. Determine yielding times `find_anticipationt_ime`
7. Solve tactical problem `solve_tactical` finds the trigger times and desired time headways based on a particular time instant where all vehicles are present in the network

In [11]:
DT = 0.1 # Sample time 

KC = 0.16 # CAV max density 
KH = 0.0896 # HDV max density
VF = 25.0 # Speed free flow
W = 6.25 # Congestion speed 
E  = 25.0*0.3 # Speed drop for relaxation 

GCAV = 1/(KC*W) # Time headway CAV 
GHDV = 1/(KH*W) # Time headway HDV 
SCAV = VF/(KC*W)+1/KC #  Desired space headway CAV 
SHDV = VF/(KH*W)+1/KH #  Desired space headway HDV

dveh_twy = {'CAV': GCAV, 'HDV': GHDV}
dveh_dwy = {'CAV': 1/KC, 'HDV': 1/KH}

U_MAX = 1.5 # Max. Acceleration
U_MIN = -1.5 # Min. Acceleration

In [12]:
def find_projection(gi, gm, w):
    """ Find the projection of point gi at speed VF
        over a point gm at speed -W
    """
    Xm, Tm = gm 
    x, t = gi
    M1 = np.array([[1,w],[1,-VF]])
    b = np.array([Xm + w * Tm, x - VF * t])
    pg = np.linalg.solve(M1,b)
    return pg 

    
def find_headway(typ_o,dveh_hwy):
    """ Determine headways for a determined sequence 
        ['CAV','HDV','CAV',...]
    """
    h_o = []
    for i,x in enumerate(typ_o):
        if i>=1:
            if typ_o[i]=='CAV' and typ_o[i-1]=='CAV':
                h_o.append(dveh_hwy['CAV'])
            else:
                h_o.append(dveh_hwy['HDV'])
        else:
            h_o.append(dveh_hwy['CAV'])
    return h_o

def find_variations(typ_o):
    """ Find all possible sortings of a-priori orders
        [0, 1, 2, 3] (Original)
        [0, 2, 1, 3] (Order 1)
        [0, 1, 3, 2] (Order 2)
    """
    var_o = [typ_o]
    for i, ty in enumerate(typ_o):
        if i>1:
            x_left = typ_o[:i]
            x_right = typ_o[i:]
            tmp = x_left[-1]
            x_left[-1] = x_right[0]
            x_right[0] = tmp
            var_o.append(x_left+x_right)
    return var_o

def find_ref_hwy(opt_typ, opt_twy, opt_dwy, opt_vid):
    """ Staring from optimized sequences find 
        reference gaps for 'CAVs'
    """
    t_ref = [(GCAV, opt_vid[0])]
    d_ref = [(0.0, opt_vid[0])]
    t_HDV = 0
    d_HDV = 0    
    for i, ty in enumerate(opt_typ):
        if i > 0:
            if ty == 'CAV' and opt_typ[i-1]=='CAV':
                t_ref.append((opt_twy[i],opt_vid[i]))
                d_ref.append((opt_dwy[i],opt_vid[i]))
            elif ty == 'CAV' and opt_typ[i-1]=='HDV':
                t_cum = t_HDV+opt_twy[i]
                d_cum = d_HDV+opt_dwy[i]
                t_ref.append((t_cum,opt_vid[i]))
                d_ref.append((d_cum,opt_vid[i]))
                t_HDV = 0
                d_HDV = 0
            else: 
                t_HDV = t_HDV + opt_twy[i] 
                d_HDV = d_HDV + opt_dwy[i]
    return t_ref,d_ref
        
def find_anticipation_time(tcav_ref, dcav_ref, results, merge_point):
    """ Compute anticipation times for CAVs
    """    
    opt_twy = [x[0] for x in tcav_ref]
    opt_dwy = [x[0] for x in dcav_ref]    
    
    T_x = np.cumsum(opt_twy)
    D_x = np.cumsum(opt_dwy)
    
    s_0 = [s[9] for s in results if s[2]=='CAV']
    
    c_hwy = (np.array(s_0) - 1/KC)/VF
    c_hwy = np.clip(c_hwy,0,np.sum(c_hwy))
    T_0 = np.cumsum(c_hwy)
    
    # Anticipation time
    T_a = E / 2 * (U_MIN-U_MAX)/ (U_MIN * U_MAX) + (VF + W) / E * (T_x - T_0)
    
    Xm, Tm = merge_point     
    
    #Projection in equilibrium
    px_eq = Xm - D_x
    pt_eq = Tm + T_x
    
    g_eq = [(x,t) for x,t in zip(px_eq,pt_eq)]    
    gm_i = [find_projection(g, merge_point, 0) for g in g_eq]
    
    tm_i = [x[1] for x in gm_i]
    xm_i = [x[0] for x in gm_i]

    # Plot projections
    p = figure(title='Projection in equilibrium')
    p.circle(pt_eq, px_eq, size = 10, color = 'blue')
    p.circle(tm_i, xm_i, size = 10, color = 'dodgerblue')
    show(p)
    

    t_s = [tf-ta for tf,ta in zip(tm_i, T_a)]
    
    c_i = [(np.round(ts,1), i[1]) for ts,i in zip(t_s, tcav_ref)]

    return c_i
    
    
def solve_tactical(merge_point, results):
    """ Tactical strategy: 
        1) Find projections
        2) Solves flow optimization problem 
        3) Determine reference headways for vehicles
    """
    g = [(x[6],x[0]) for x in results]
    ty = [x[2] for x in results]
    vid = [x[1] for x in results]

    pgt = []
    pgx = []
    
    for gi in g:
        pg  = find_projection(gi, merge_point, W)        
        pgx.append(pg[0])
        pgt.append(pg[1])
    
    
    # Plot order 
    dc = {'CAV':'blue','HDV':'red'}
    col = [dc[x] for x in ty]
    p = figure(title='Projection')
    p.xaxis.axis_label = 'Time [s]'
    p.yaxis.axis_label = 'Space [m]'
    p.circle(pgt, pgx, color = col, size = 10)
    show(p)

    
    # Sort according 
    vid_o1 = [x for _,x in sorted(zip(pgt,vid))]
    typ_o1 = [x for _,x in sorted(zip(pgt,ty))]
    
    # Find variations
    var_o1 = find_variations(typ_o1)  
    var_vid_o1 = find_variations(vid_o1) 
    
    h_var_o1 = [find_headway(x, dveh_twy) for x in var_o1]
    tot_var_o1 = [sum(x) for x in h_var_o1]
    
    # Optimize     
    opt_idx = tot_var_o1.index(min(tot_var_o1))
    opt_seq = var_o1[opt_idx]    
    opt_vid = var_vid_o1[opt_idx]
    
    # Apriori 
    apr_idx = 0 
    apr_seq = var_o1[apr_idx]
    apr_vid = var_vid_o1[apr_idx]
    
#     print(f'Apriori sequence: {apr_idx}')  
#     print(f'Optimal sequence: {opt_idx}')    

#     print(f'A-priori order: {apr_seq}')
#     print(f'Apriori vehicle index: {apr_vid}')  
#     print(f'Optimized order:{opt_seq}')
#     print(f'Optimal vehicle index: {opt_vid}') 
    
    # Find new gaps 
    tapr_seq = find_headway(apr_seq, dveh_twy)
    dapr_seq = find_headway(apr_seq, dveh_dwy)
    
    topt_seq = find_headway(opt_seq, dveh_twy)
    dopt_seq = find_headway(opt_seq, dveh_dwy)
    
#     print(f'A-priori headway:\n {tapr_seq}')
#     print(f'Optimized headway:\n {topt_seq}')
    
    hcav_ref, dcav_ref = find_ref_hwy(opt_seq, topt_seq, dopt_seq, opt_vid)
#     print(f'Reference equilibirum CAV (time):\n {hcav_ref}')
#     print(f'Reference equilibrium CAV (space):\n {dcav_ref}')
    
    # Find anticipation times 
    tcav_ref = find_anticipation_time(hcav_ref, dcav_ref, results, merge_point)
#     print(f'Trigger times: \n {tcav_ref}')
    
    return (opt_seq,opt_vid,hcav_ref,tcav_ref)
    

<a id='yielding_time'></a>
#### Solve tactical layer 

In [13]:
# Query at moment where all vehicles are in the network  
t_i = 12.5
stmt = select([traj]).where(traj.columns.ti==t_i)
results = connection.execute(stmt).fetchall()
results

[(12.5, 0, 'CAV', 'In_main', 1, 281.25, -718.75, 25.0, 0, 31.25, 25.0),
 (12.5, 1, 'CAV', 'In_main', 1, 250.0, -750.0, 25.0, 0, 31.25, 25.0),
 (12.5, 2, 'CAV', 'In_main', 1, 218.75, -781.25, 25.0, 1, 31.25, 25.0),
 (12.5, 3, 'CAV', 'In_main', 1, 187.5, -812.5, 25.0, 2, 31.25, 25.0),
 (12.5, 4, 'HDV', 'In_onramp', 1, 164.11, -736.59, 25.0, 4, 55.80357142857143, 25.0),
 (12.5, 5, 'CAV', 'In_main', 1, 156.25, -843.75, 25.0, 3, 31.25, 25.0),
 (12.5, 6, 'CAV', 'In_main', 1, 125.0, -875.0, 25.0, 5, 31.25, 25.0),
 (12.5, 7, 'HDV', 'In_onramp', 1, 108.3, -792.15, 25.0, 4, 55.559999999999945, 25.0),
 (12.5, 8, 'CAV', 'In_main', 1, 93.75, -906.25, 25.0, 6, 31.25, 25.0),
 (12.5, 9, 'CAV', 'In_main', 1, 62.5, -937.5, 25.0, 8, 31.25, 25.0),
 (12.5, 10, 'HDV', 'In_onramp', 1, 52.5, -847.72, 25.0, 7, 55.57000000000005, 25.0)]

In [14]:
a = solve_tactical(merge_point,results)
d_ev = {ev[0]:(ev[1],tr[0]) for ev,tr in zip(a[-1],a[-2])}
print('Event: (id, value of gap to open)')
d_ev

Event: (id, value of gap to open)


{21.0: (6, 3.5714285714285716),
 22.2: (8, 1.0),
 23.5: (9, 1.0),
 25.9: (3, 3.5714285714285716),
 27.2: (5, 1.0),
 31.0: (1, 3.5714285714285716),
 32.2: (2, 1.0),
 37.2: (0, 1.0)}

<a id='reference_headway'></a>
### 2b. Tactical to operational layer 

In this case starting from `d_ev` which is the dictionary containing the time instant the id of the vehicle and the final reference gap, the desired time headway is found 

In [15]:
def headway_reference(gap_events):
    """ Determine the time signal for the reference 
        of the controller. 
    """
    stmt = delete(headway) 
    connection.execute(stmt)
    
    ti = trajDf.ti.unique()
    hr = []
    
    for k, v in gap_events.items(): 
        if k == 0: 
            ref = (v[1]-1.0) / (1 + np.exp(-(ti-k)))
        else: 
            ref = 1.0 + (v[1]-1.0) / (1 + np.exp(-(ti-k)))
        ref = np.clip(ref, 1.0, 100)
        hr.append((ref,v[0])) 

    
    stmt = insert(headway)  
    
    for i,v in enumerate(hr):
        lRefVeh = []
        for k, t_i in enumerate(ti):
            # Tests Time headway            
            # Case1 : FORCED (Equilibirum)
#             rvehti = {'ti': t_i, 'id':v[1], 'gapt': GCAV} # Forced 
            # Case 2: FORCED (Small)
#             if v[1] == 1:
#                 rvehti = {'ti': t_i, 'id':v[1], 'gapt': GCAV + 0.1} # Forced 
#             else:
#                 rvehti = {'ti': t_i, 'id':v[1], 'gapt': GCAV} # Forced             
            # Case 3: REAL GAP
            rvehti = {'ti': t_i, 'id':v[1], 'gapt':v[0][k]}
            lRefVeh.append(rvehti)
        results = connection.execute(stmt,lRefVeh)
   
    
    print('Headway table written')
    return (ti,hr)

ti,hr = headway_reference(d_ev)

Headway table written


#### Reading from dB

Reading `headway` table to check reference headway times 

In [16]:
stmt = select([headway])

results = connection.execute(stmt).fetchall()

column_names = headway.columns.keys()
headDf = pd.DataFrame(results, columns = column_names)
headDf.head()


veh_id = headDf.id.unique()
key_id = [k for k in veh_id]
color_id = Paired[12]+Paired[12]
color_id = [x for i,x in enumerate(color_id) if i%2 ==0]
colormapid = dict(zip(key_id,color_id))
color_vec =  [colormapid[x] for x in headDf.id]

p = figure(title='Reference Gap')
p.xaxis.axis_label = 'Time [s]'
p.yaxis.axis_label = 'Tau [s]'
p.circle(headDf.ti, headDf.gapt, color = color_vec , size = 2)
show(p)